In [1]:
# !pip install rio-tiler
# !pip install ipyleaflet

In [1]:
import morecantile
from ipyleaflet import Map, TileLayer, projections

In [2]:
# For this DEMO we will use this file
src_path = "https://njogis-imagery.s3.amazonaws.com/2020/cog/I7D16.tif"

In [3]:
from concurrent import futures

from tornado import gen, web
from tornado.concurrent import run_on_executor
from tornado.httpserver import HTTPServer

from rio_tiler.errors import TileOutsideBounds
from rio_tiler.io import Reader
from rio_tiler.profiles import img_profiles


class TileServer:
    def __init__(self, src_path):
        """Initialize Tornado app."""
        self.server = None
        self.app = web.Application(
            [
                (r"^/tiles/(\w+)/(\d+)/(\d+)/(\d+)", TileHandler, {"url": src_path}),
            ]
        )

    def start(self):
        """Start tile server."""
        self.server = HTTPServer(self.app)
        self.server.listen(8080)

    def stop(self):
        """Stop tile server."""
        if self.server:
            self.server.stop()


class TileHandler(web.RequestHandler):
    """Tile requests handler."""

    executor = futures.ThreadPoolExecutor(max_workers=16)

    def initialize(self, url):
        """Initialize tiles handler."""
        self.url = url

    @run_on_executor
    def _get_tile(self, tms, z, x, y):
        try:
            with Reader(self.url, tms=morecantile.tms.get(tms)) as cog:
                img = cog.tile(x, y, z, indexes=(1, 2, 3))
        except TileOutsideBounds:
            raise web.HTTPError(404)

        prof = img_profiles.get("PNG", {})
        return img.render(img_format="PNG", **prof)

    @gen.coroutine
    def get(self, tms, z, x, y):
        """Retunrs tile data and header."""
        self.set_header("Access-Control-Allow-Origin", "*")
        self.set_header("Access-Control-Allow-Methods", "GET")
        self.set_header("Content-Type", "image/png")
        self.set_header("Cache-Control", "no-store, no-cache, must-revalidate")
        res = yield self._get_tile(tms, int(z), int(x), int(y))
        self.write(res)


ts = TileServer(src_path)
ts.start()

In [4]:
print("Supported TMS:")
for name in morecantile.tms.list():
    print("-", name)

Supported TMS:
- CanadianNAD83_LCC
- CDB1GlobalGrid
- EuropeanETRS89_LAEAQuad
- GNOSISGlobalGrid
- LINZAntarticaMapTilegrid
- NZTM2000Quad
- UPSAntarcticWGS84Quad
- UPSArcticWGS84Quad
- UTM31WGS84Quad
- WebMercatorQuad
- WGS1984Quad
- WorldCRS84Quad
- WorldMercatorWGS84Quad


In [5]:
src_path = "D:\\EmDrone\\Lahad Datu\\lahad_datu_cog.tif"
# src_path = "C:\\Users\\Nicklim\\Documents\\Personal\\Projects\\Medium\\OGC\\pygeo-server\\data\\mersing-rgb.tiff"
with Reader(src_path) as cog:
    info = cog.info()
    minzoom = cog.minzoom
    maxzoom = cog.maxzoom
    bounds = cog.get_geographic_bounds(cog.tms.rasterio_geographic_crs)

print(info.model_dump(exclude_none=True))
print(bounds)
print(minzoom, maxzoom)

center = ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2)
m = Map(center=center, zoom=minzoom, basemap={})

layer = TileLayer(
    url="http://127.0.0.1:8000/api/tiles/lahad_datu/{z}/{x}/{y}",
    min_zoom=minzoom,
    max_zoom=maxzoom+2,
    opacity=1,
)
m.add_layer(layer)
m

{'bounds': (2681963.3968851496, 591777.0659296932, 2684162.5989161497, 593574.0311672932), 'crs': 'http://www.opengis.net/def/crs/EPSG/0/32647', 'band_metadata': [('b1', {'STATISTICS_APPROXIMATE': 'YES', 'STATISTICS_MAXIMUM': '232', 'STATISTICS_MEAN': '93.427102265171', 'STATISTICS_MINIMUM': '41', 'STATISTICS_STDDEV': '23.414576247576', 'STATISTICS_VALID_PERCENT': '70.33'}), ('b2', {'STATISTICS_APPROXIMATE': 'YES', 'STATISTICS_MAXIMUM': '228', 'STATISTICS_MEAN': '133.97841216366', 'STATISTICS_MINIMUM': '65', 'STATISTICS_STDDEV': '18.587782907648', 'STATISTICS_VALID_PERCENT': '70.33'}), ('b3', {'STATISTICS_APPROXIMATE': 'YES', 'STATISTICS_MAXIMUM': '222', 'STATISTICS_MEAN': '84.303648211357', 'STATISTICS_MINIMUM': '43', 'STATISTICS_STDDEV': '17.222671627944', 'STATISTICS_VALID_PERCENT': '70.33'}), ('b4', {'STATISTICS_APPROXIMATE': 'YES', 'STATISTICS_MAXIMUM': '255', 'STATISTICS_MEAN': '178.42586797449', 'STATISTICS_MINIMUM': '0', 'STATISTICS_STDDEV': '116.88800610432', 'STATISTICS_VALID

Map(center=[5.059510775360735, 118.32144113207127], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [1]:
with Reader(src_path, tms=morecantile.tms.get("WorldCRS84Quad")) as cog:
    info = cog.info()
    minzoom = cog.minzoom
    maxzoom = cog.maxzoom
    bounds = cog.get_geographic_bounds(cog.tms.rasterio_geographic_crs)

print(info.model_dump(exclude_none=True))

center = ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2)
m = Map(center=center, zoom=minzoom, basemap={}, crs=projections.EPSG4326)

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WorldCRS84Quad/{z}/{x}/{y}",
    min_zoom=minzoom,
    max_zoom=maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

NameError: name 'Reader' is not defined

In [ ]:
ts.stop()